In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm.auto import tqdm
import torchvision.models
import cv2
import matplotlib.style as mplstyle
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns

In [ ]:
# hyperparams

batch_size = 32 
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 8
lr = 1.9939377918517657e-05
clr = 0.00031384783419999377
dropout = 0.3
weight_decay = 3.2730287919730626e-05
features = -4

In [ ]:
imagenet_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    imagenet_normalize
])


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    imagenet_normalize
])


In [ ]:
train_dataset = datasets.ImageFolder(root=r"C:\Users\HP\Desktop\phone-detector\Train-6", transform=train_transforms)
test_dataset  = datasets.ImageFolder(root=r"C:\Users\HP\Desktop\phone-detector\Test-2",  transform=test_transforms)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
print(train_dataset.class_to_idx)  
print(test_dataset.class_to_idx)

In [ ]:
model = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1)

for param in model.features[features:].parameters():
    param.requires_grad = True

print(model.features[features:])

model.classifier = nn.Sequential(
    nn.Linear(25088, 512),
    nn.ReLU(),
    nn.Dropout(dropout),
    nn.Linear(512, 1)  
)

model.to(device)

In [ ]:
optimizer = torch.optim.AdamW([
    {"params": model.features[features:].parameters(), "lr": lr},  
    {"params": model.classifier.parameters(), "lr": clr},     
], weight_decay=weight_decay)


loss_fn = torch.nn.BCEWithLogitsLoss()


In [ ]:
train_losses = []
test_losses = []

train_accs = []
test_accs = []

def train_loop(epochs, loader, model, loss_fn, optimizer):
    total_batches = len(loader)

    for epoch in tqdm(range(epochs)):

        total_epoch_loss = 0
        total_acc = 0
        print(f"\n -------- Epoch {epoch} --------")\
        
        for batch, (X, Y) in enumerate(loader):
            model.train()

            X, Y = X.to(device), Y.to(device)

            Y_pred = model(X)
            Y = Y.unsqueeze(1).float()
            loss = loss_fn(Y_pred, Y)
            correct = (Y_pred == Y).sum().item()
            total_acc += correct


            total_epoch_loss += loss.item()
            

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_acc = total_acc / len(loader.dataset)

        total_epoch_loss /= total_batches
        print()
        print(f"Avg training loss: {total_epoch_loss:.2f} | Average accuracy {total_acc:.2f}")
        train_losses.append(total_epoch_loss)
        train_accs.append(total_acc)


def test_loop(loader, model, loss_fn):
    
    total_batches = len(loader)

    model.eval()
    total_loss, total_acc = 0, 0
    all_preds = []
    all_labels = []
    with torch.inference_mode():

        for X, Y in loader:
            
            X, Y = X.to(device), Y.to(device)

            Y_pred = model(X)
            Y = Y.unsqueeze(1).float()
            loss = loss_fn(Y_pred, Y)

            total_loss += loss.item()
            

            probs = torch.sigmoid(Y_pred)
            preds = (probs > 0.5).float()
            correct = (preds == Y).sum().item()
            total_acc += correct

            all_preds.extend(preds.cpu().numpy().flatten())
            all_labels.extend(Y.cpu().numpy().flatten())

        total_loss /= total_batches
        total_acc = total_acc / len(loader.dataset)

    precision = precision_score(all_labels, all_preds, zero_division=0)
    recall = recall_score(all_labels, all_preds, zero_division=0)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    
    print(f"Test loss: {total_loss:.2f} | Test accuracy: {total_acc:.2f}")
    print(f"Precision: {precision:.2f} | Recall: {recall:.2f} | F1: {f1:.2f}")
    print(f"\nConfusion Matrix:\n{cm}")
    
    test_losses.append(total_loss)
    test_accs.append(total_acc)
    
   
    return {
        'loss': total_loss,
        'accuracy': total_acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': cm,
        'preds': all_preds,
        'labels': all_labels
    }
        


In [ ]:
train_loop(epochs, train_loader, model, loss_fn, optimizer)

In [ ]:
test_metrics = test_loop(test_loader, model, loss_fn)

In [ ]:
print(plt.style.available)

In [ ]:
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_losses, marker='o', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:

plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

metrics_dict = test_metrics
print(metrics_dict)
metric_names = ['Precision', 'Recall', 'F1 Score']
metric_values = [metrics_dict['precision'], metrics_dict['recall'], metrics_dict['f1']]

plt.figure(figsize=(8, 6))
bars = plt.bar(metric_names, metric_values, color=['#3498db', '#2ecc71', '#e74c3c'])
plt.ylabel('Score')
plt.ylim(0, 1)
plt.title('Classification Metrics on Test Set')
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, metric_values):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{value:.3f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')


cm = metrics_dict['confusion_matrix']

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Phone', 'Phone'],
            yticklabels=['No Phone', 'Phone'],
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix: Phone Detection Model')
plt.tight_layout()
plt.show()


tn, fp, fn, tp = cm.ravel()
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

In [ ]:
def live_pred(model):
    cam = cv2.VideoCapture(0)

    while True:

        ret, frame = cam.read()
        if not ret:
            return

        frame = cv2.resize(frame, (224, 224))

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        tensor = torch.from_numpy(rgb_frame).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        tensor = tensor.to(device)

        mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
        std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)
        tensor = (tensor - mean) / std

        with torch.no_grad():
            pred = model(tensor)

            probs = torch.sigmoid(pred)
            prob = probs.item()


        print(f"Prediciton: {1 if prob >= 0.9 else 0} | Actual item: {prob}")
        if cv2.waitKey(1) & 0XFF == ord('q'):
            break

        cv2.imshow("Live feed", frame)

    cam.release()
    cv2.destroyAllWindows()
    return 

In [ ]:
live_pred(model)